<a href="https://colab.research.google.com/github/SeanBarnier/HAFS_Air-Sea/blob/main/miltonAnimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cartopy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cft
import pandas as pd
from datetime import datetime as dt

In [ ]:
name = "Milton"
tcNum = "14"
trackType = ""

initTime = dt(year=2024, month=10, day=7, hour=0) #Time when Milton began its most rapid intensification
initStr, initHour = initTime.strftime("%Y%m%d_%H").split("_")

fHourStep = 12      #Normally 3 for HAFS-A
forecastLength = 54 #Normally 126 for HAFS-A.
#runStep = 6         #Normally 6 for HAFS-A

figureSuffix = "_RI"
subfolder = "RI/"
dataPath = "/content/drive/MyDrive/savedData/"
figurePath = "/content/drive/MyDrive/figures/"

potentialTemp = True #Use atmospheric potential temperature instead of in-situ temperature

lonMin, lonMax, latMin, latMax = -95, -85, 21, 26
atmLayer = 1000.0

Find times needed

In [ ]:
dateFormat = "%Y-%m-%d %H:%M:%S"
runFormat = "%Y%m%d%H"

#Times to plot
fcastTimes = [] #Key: initiation, item: valid time list
fhour = 0
validTime = initTime
while fhour <= forecastLength:
    fcastTimes.append(validTime)
    validTime += pd.Timedelta(hours=fHourStep)
    fhour += fHourStep

In [ ]:
atmFig, atmAx = plt.subplots()
xdata, ydata = [], []

contourLevs = np.linspace(0, 80, 17)

atmPlot = atmAx.contourf([1,1], [1,1], [[1,1], [1,1]])

def update(valid):

  fhour = str(int((valid-initTime).total_seconds() / 3600))
  while len(fhour) < 3: fhour = "0" + fhour

  atmFile = "hafsa_" + initStr + initHour + "_f" + fhour + ".nc"
  atmPath = dataPath + "hafsaOutput/" + subfolder + atmFile
  atmData = xr.open_dataset(atmPath)

  atmSlice = atmData.isel(isobaricInhPa=atmLayer).sel(longitude=slice(lonMin+360, lonMax+360), latitude=slice(latMin, latMax))
  dat = np.sqrt(atmSlice.u.data ** 2 + atmSlice.v.data ** 2)

  #if "Empty" in contourLevs[atmLayer]: contourLevs[atmLayer] = [round(l, 1) for l in np.linspace(np.min(temp[np.isnan(temp)==False])-3, np.max(temp[np.isnan(temp)==False])+3, 15)]

  tempContour = atmAx.contourf(atmSlice.longitude.data, atmSlice.latitude.data, dat, cmap="viridis", transform=ccrs.PlateCarree(),
                              extent = [lonMin, lonMax, latMin, latMax], levels=contourLevs)

  if valid == fcastTimes[-1]: atmFig.colorbar(tempContour, shrink=0.8)

  #atmAx.scatter(tcLocs[valid][1], tcLocs[valid][0], marker="*", color="black", s=75, transform=ccrs.PlateCarree())
  #atmAx.add_feature(cft.COASTLINE)
  #atmAx.add_feature(cft.BORDERS)
  #atmAx.gridlines(draw_labels=["left", "bottom"], alpha=0.5)
  #atmAx.set_title(f'{atmData.isobaricInhPa.data[atmLayer]} hPa\n{valid.strftime("%Y-%m-%d %HUTC")}')

  atmAx.set_extent([lonMin, lonMax, latMin, latMax])

  atmPlot.set_data(xdata, ydata)
  return atmPlot

def init():
  #ax.set_xlim(0, 2*np.pi)
  #ax.set_ylim(-1, 1)
  return update(fcastTimes[0])

ani = FuncAnimation(atmFig, update, frames=fcastTimes,
                    init_func=init, blit=True)
plt.show()

In [ ]:
ani